<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=ec3f21d4a7aaa7afeb19247e89350d44e019072b6be2703f7dd083fbd7f11262
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1


    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-20 09:29:06
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.16 L
Current:  1.45 C
-------------------
Today PnL: 7.49 K (0.05%)
Current PnL: -21.16 L (-13.87%)
CY Booked + Current PnL: -7.20 L (-4.72%)
-------------------
Total profit:  1.65 L
Total loss:  -22.81 L
-------------------
Total Booked + Current PnL: 19.78 L (15.74%)
Total Booked PnL: 40.94 L (32.59%)
Curr Year Booked PnL: 13.96 L (9.65%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 86.08 L (59.52%)
Deployed:  1.26 C
Current:  1.45 C
CAGR/XIRR %: 8.1%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,0.65,-7.51,8.15,0.02,13158.0,-13117.0,161442.0,146.09,77.0,M-SC,3.04,234.0,-1.00,1.13,40.43,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,-0.07,-14.14,16.58,0.09,14346.0,-14251.0,86526.0,100.85,50.0,M-SC,4.04,253.0,-0.99,0.60,13.68,OX40N,NTT,DURABLES
83,VOLTAS,1530.00,-0.20,9.44,9.36,19.69,19642.0,18108.0,209850.0,-1.22,55.0,X-MC,3.63,78.0,0.92,1.46,16.88,XY25,NTT,AC
35,ICICIGI,2252.93,-0.17,8.74,10.53,20.19,19961.0,15242.0,189562.0,-14.84,57.0,X-MC,7.69,68.0,0.76,1.32,25.18,X40,ATH,INSURANCE
2,ABBOTINDIA,35195.00,0.22,-1.59,18.92,17.03,22397.0,-1912.0,118380.0,-14.93,49.0,X-MC,4.00,63.0,-0.09,0.83,17.67,X40,ATH,PHARMA


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,3.90,-34.00,71.07,12.90,102487.0,-74290.0,144206.0,112.74,65.0,H-SC,15.62,161.0,-0.72,1.01,19.91,OX40N,NTT,FINANCE
57,RAJESHEXPO,518.00,2.71,-62.53,167.07,0.07,85872.0,-85778.0,51399.0,1783.84,67.0,L-SC,2.18,268.0,-1.00,0.36,27.83,OX40N,NTT,JEWELLERY
26,GILLETTE,11206.78,1.04,-0.40,32.96,32.42,83346.0,-1026.0,252870.0,-18.02,34.0,X-SC,6.17,85.0,-0.01,1.76,14.63,X40,ATH,FMCG
23,DMART,5391.45,0.81,-3.54,32.83,28.12,66628.0,-7457.0,202950.0,-16.76,38.0,X-LC,5.11,19.0,-0.11,1.41,19.24,X40N,ATH,FMCG
15,CAMS,4762.00,0.79,3.77,20.25,24.79,52925.0,9504.0,261360.0,-1.02,57.0,X-SC,2.17,86.0,0.18,1.82,27.79,X40N,NTT,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VALIANTORG,838.0,-2.35,-43.48,208.14,74.15,192452.0,-71142.0,92463.0,-266.56,30.0,H-SC,13.31,149.0,-0.37,0.64,18.52,XR,NTT,CHEMICALS
80,VAIBHAVGBL,521.0,-1.65,-23.70,110.72,60.77,154398.0,-43326.0,139449.0,60.88,46.0,H-SC,11.06,159.0,-0.28,0.97,25.58,XR,NTT,JEWELLERY
13,BSOFT,831.7,-1.14,-29.86,114.99,50.79,113434.0,-42006.0,98647.0,-9.93,57.0,H-SC,7.91,151.0,-0.37,0.69,14.86,XR,ATH,IT
6,ASIANTILES,137.0,-1.01,-11.82,106.02,81.67,87706.0,-11084.0,82726.0,7438.89,55.0,L-SC,18.11,271.0,-0.13,0.58,60.20,XR,NTT,CERAMICS
12,BERGEPAINT,680.0,-0.88,3.99,16.49,21.14,38985.0,9080.0,236419.0,-3.37,69.0,X-MC,3.26,74.0,0.23,1.65,33.26,XY24,NTT,PAINTS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.17,-1.92,111.98,107.91,162162.0,-2831.0,144813.0,-20.15,42.0,M-SC,10.11,216.0,-0.02,1.01,1.02,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.65,-7.51,8.15,0.02,13158.0,-13117.0,161442.0,146.09,77.0,M-SC,3.04,234.0,-1.00,1.13,40.43,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,-0.07,-14.14,16.58,0.09,14346.0,-14251.0,86526.0,100.85,50.0,M-SC,4.04,253.0,-0.99,0.60,13.68,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-0.72,-18.87,39.86,13.47,87210.0,-50877.0,218790.0,-66.56,40.0,H-SC,1.76,158.0,-0.58,1.52,11.25,XY24,NTT,PAINTS
73,TATAELXSI,9161.00,0.01,-22.92,71.30,32.03,72449.0,-30223.0,101612.0,-16.54,48.0,H-SC,6.41,148.0,-0.42,0.71,13.81,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,0.10,1.17,70.51,72.51,107681.0,1773.0,152718.0,-13.34,56.0,M-LC,5.08,99.0,0.02,1.06,7.27,XR,NTT,IT
38,INDIAMART,4810.62,0.19,3.91,98.95,106.72,126808.0,4818.0,128154.0,-48.53,45.0,H-SC,4.62,139.0,0.04,0.89,28.20,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,0.19,3.91,98.95,106.72,126808.0,4818.0,128154.0,-48.53,45.0,H-SC,4.62,139.0,0.04,0.89,28.20,AR,ATH,MISC
85,WIPRO,420.00,0.10,1.17,70.51,72.51,107681.0,1773.0,152718.0,-13.34,56.0,M-LC,5.08,99.0,0.02,1.06,7.27,XR,NTT,IT
25,FINCABLES,1641.55,-0.17,-1.92,111.98,107.91,162162.0,-2831.0,144813.0,-20.15,42.0,M-SC,10.11,216.0,-0.02,1.01,1.02,OX40N,ATH,CABLES
86,ZYDUSLIFE,1286.17,-0.08,-1.99,38.38,35.63,78836.0,-4166.0,205408.0,-15.85,33.0,H-MC,5.01,119.0,-0.05,1.43,14.35,AR,ATH,PHARMA
84,WHIRLPOOL,2270.00,-0.39,-4.04,89.58,81.92,98682.0,-4637.0,110161.0,-43.78,34.0,M-SC,6.99,236.0,-0.05,0.77,30.41,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.00,0.68,-13.91,65.15,42.18,155032.0,-38448.0,237962.0,-23.08,24.0,X-LC,0.01,3.0,-0.25,1.66,0.68,XY24,NTT,AUTO
11,BATAINDIA,2096.00,0.50,-34.40,106.10,35.20,89560.0,-44259.0,84411.0,10.61,27.0,X-SC,11.45,92.0,-0.49,0.59,0.50,X40,NTT,FOOTWEAR
34,HONAUT,58357.33,0.26,-18.20,67.28,36.84,70412.0,-23283.0,104655.0,-26.34,30.0,X-SC,1.43,90.0,-0.33,0.73,7.14,X40N,ATH,ELECTRICAL
60,RELAXO,1176.00,-0.36,-46.63,185.37,52.31,143615.0,-67685.0,77475.0,-43.18,31.0,X-SC,3.29,91.0,-0.47,0.54,3.70,X40N,NTT,FOOTWEAR
9,BAJAJHFL,181.50,0.24,-13.05,70.73,48.45,123316.0,-26158.0,174348.0,-19.23,32.0,X-MC,7.09,64.0,-0.21,1.22,0.54,X40N,ATH,FINANCE


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.00,0.68,-13.91,65.15,42.18,155032.0,-38448.0,237962.0,-23.08,24.0,X-LC,0.01,3.0,-0.25,1.66,0.68,XY24,NTT,AUTO
43,ITC,452.00,0.32,-1.08,11.65,10.44,23064.0,-2166.0,197972.0,-40.74,44.0,X-LC,0.94,5.0,-0.09,1.38,4.42,X40,NTT,FMCG
36,ICICIPRULI,790.00,-0.39,1.38,29.18,30.97,52286.0,2446.0,179184.0,-22.52,50.0,X-MC,1.41,75.0,0.05,1.25,14.12,X40,ATH,INSURANCE
34,HONAUT,58357.33,0.26,-18.20,67.28,36.84,70412.0,-23283.0,104655.0,-26.34,30.0,X-SC,1.43,90.0,-0.33,0.73,7.14,X40N,ATH,ELECTRICAL
66,SIEMENS,4671.50,0.70,-13.11,44.45,25.51,71876.0,-24395.0,161700.0,-0.76,61.0,H-LC,1.83,49.0,-0.34,1.13,19.16,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,PAGEIND,51605.07,0.26,-4.95,32.63,26.07,50785.0,-8100.0,155640.0,-29.94,34.0,X-MC,8.42,55.0,-0.16,1.08,0.26,X40,ATH,APPARELS
11,BATAINDIA,2096.00,0.50,-34.40,106.10,35.20,89560.0,-44259.0,84411.0,10.61,27.0,X-SC,11.45,92.0,-0.49,0.59,0.50,X40,NTT,FOOTWEAR
9,BAJAJHFL,181.50,0.24,-13.05,70.73,48.45,123316.0,-26158.0,174348.0,-19.23,32.0,X-MC,7.09,64.0,-0.21,1.22,0.54,X40N,ATH,FINANCE
76,TMPV,600.00,0.68,-13.91,65.15,42.18,155032.0,-38448.0,237962.0,-23.08,24.0,X-LC,0.01,3.0,-0.25,1.66,0.68,XY24,NTT,AUTO
55,PGHH,17907.65,0.34,-3.28,38.29,33.76,74373.0,-6585.0,194235.0,-31.71,36.0,X-MC,2.86,57.0,-0.09,1.35,1.75,X40,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-0.60,-25.32,91.28,42.84,44311.0,-16462.0,48544.0,-50.96,43.0,X-SC,37.09,83.0,-0.37,0.34,5.18,XY24,NTT,MISC
60,RELAXO,1176.00,-0.36,-46.63,185.37,52.31,143615.0,-67685.0,77475.0,-43.18,31.0,X-SC,3.29,91.0,-0.47,0.54,3.70,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.50,-34.40,106.10,35.20,89560.0,-44259.0,84411.0,10.61,27.0,X-SC,11.45,92.0,-0.49,0.59,0.50,X40,NTT,FOOTWEAR
34,HONAUT,58357.33,0.26,-18.20,67.28,36.84,70412.0,-23283.0,104655.0,-26.34,30.0,X-SC,1.43,90.0,-0.33,0.73,7.14,X40N,ATH,ELECTRICAL
52,MEDANTA,1486.00,-0.61,-0.50,24.70,24.07,27080.0,-551.0,109636.0,-8.54,37.0,X-SC,4.74,89.0,-0.02,0.76,19.02,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.96,0.04,-14.44,39.41,19.28,116656.0,-49950.0,296006.0,-25.57,63.0,X-LC,7.94,1.0,-0.43,2.06,9.43,X40,ATH,IT
41,INFY,2275.00,-0.31,4.59,48.07,54.87,157311.0,14365.0,327253.0,-18.52,58.0,X-LC,3.42,2.0,0.09,2.28,13.25,X40,BTT,IT
76,TMPV,600.00,0.68,-13.91,65.15,42.18,155032.0,-38448.0,237962.0,-23.08,24.0,X-LC,0.01,3.0,-0.25,1.66,0.68,XY24,NTT,AUTO
82,VBL,671.64,-0.06,-8.23,47.84,35.67,138661.0,-25999.0,289843.0,-19.21,45.0,X-LC,6.00,4.0,-0.19,2.02,4.48,X40N,ATH,FMCG
43,ITC,452.00,0.32,-1.08,11.65,10.44,23064.0,-2166.0,197972.0,-40.74,44.0,X-LC,0.94,5.0,-0.09,1.38,4.42,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,-1.01,-11.82,106.02,81.67,87706.0,-11084.0,82726.0,7438.89,55.0,L-SC,18.11,271.0,-0.13,0.58,60.20,XR,NTT,CERAMICS
68,SONACOMS,806.63,-0.63,-12.85,60.13,39.54,53008.0,-13003.0,88156.0,-31.83,67.0,M-SC,6.81,220.0,-0.25,0.61,24.31,AR,ATH,AUTO
51,MASFIN,398.61,0.33,-3.52,26.50,22.05,25050.0,-3450.0,94530.0,-16.89,56.0,H-SC,7.22,164.0,-0.14,0.66,37.41,XR,ATH,FINANCE
80,VAIBHAVGBL,521.00,-1.65,-23.70,110.72,60.77,154398.0,-43326.0,139449.0,60.88,46.0,H-SC,11.06,159.0,-0.28,0.97,25.58,XR,NTT,JEWELLERY
70,SURYODAY,216.00,-0.48,-21.24,56.22,23.03,79290.0,-38036.0,141035.0,53.51,44.0,H-SC,12.25,167.0,-0.48,0.98,39.94,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,0.65,-7.51,8.15,0.02,13158.0,-13117.0,161442.0,146.09,77.0,M-SC,3.04,234.0,-1.00,1.13,40.43,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,-0.06,1.09,19.64,20.95,49985.0,2737.0,254505.0,0.07,81.0,X-LC,11.49,31.0,0.05,1.77,37.11,X40,ATH,PAINTS
13,BSOFT,831.70,-1.14,-29.86,114.99,50.79,113434.0,-42006.0,98647.0,-9.93,57.0,H-SC,7.91,151.0,-0.37,0.69,14.86,XR,ATH,IT
0,5PAISA,593.00,3.90,-34.00,71.07,12.90,102487.0,-74290.0,144206.0,112.74,65.0,H-SC,15.62,161.0,-0.72,1.01,19.91,OX40N,NTT,FINANCE
80,VAIBHAVGBL,521.00,-1.65,-23.70,110.72,60.77,154398.0,-43326.0,139449.0,60.88,46.0,H-SC,11.06,159.0,-0.28,0.97,25.58,XR,NTT,JEWELLERY


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.85
1,25,44.87
2,50,75.75


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.82
MC    29.09
LC    25.06
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.10
X40      20.98
X40N     11.60
XR        9.52
AR        9.15
XY25      9.13
OX40N     7.79
X200      1.88
SR        1.01
MH        0.81
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.40
X-MC    22.63
X-LC    20.01
M-SC    12.16
X-SC     7.83
H-MC     4.75
H-LC     3.01
L-SC     1.43
M-MC     1.38
M-LC     1.06
L-LC     0.98
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.81,-4.26,37.98
IT,13.68,-13.39,71.98
FINANCE,9.52,-13.61,61.62
MISC,7.49,-20.97,70.53
PAINTS,6.07,-5.99,21.73
ELECTRICAL,5.68,-10.87,50.17
INSURANCE,4.50,1.98,32.57
PHARMA,3.66,-0.83,33.83
AUTO,3.36,-19.10,70.17


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2983618.0,21
XR,1263271.0,13
AR,1258949.0,10
X40,990532.0,14
X40N,706689.0,9
OX40N,676995.0,10
XY25,330306.0,6
SR,275514.0,2
MH,67612.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3477364.0,25
M-SC,1313974.0,15
X-MC,1181522.0,16
X-LC,889431.0,11
X-SC,717075.0,8
H-MC,396685.0,3
L-SC,256907.0,3
H-LC,126137.0,2
M-LC,107681.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1184986.0      6
           AR         851377.0      5
           XR         783209.0      7
M-SC       XY24       769124.0      6
X-MC       X40        427626.0      7
X-LC       X40        390000.0      5
X-MC       XY24       345350.0      3
H-SC       OX40N      314666.0      4
X-SC       XY24       277217.0      3
M-SC       OX40N      276457.0      5
H-SC       SR         275514.0      2
X-SC       X40N       266952.0      3
X-MC       X40N       234448.0      4
X-LC       XY24       224209.0      2
H-MC       AR         213953.0      2
X-LC       X40N       205289.0      2
H-MC       XY24       182732.0      1
X-MC       XY25       174098.0      2
X-SC       X40        172906.0      2
L-SC       XR         171035.0      2
M-SC       XR         146650.0      2
           AR         121743.0      2
M-LC       XR         107681.0      1
L-SC       OX40N       85872.0      1
H-LC       AR          71876.0      1
X-LC       XY25        69933.0      2
H-SC       MH          67612.0      1
L-MC       XR          54696.0      1
H-LC       X200        54261.0      1
M-MC       XY25        47077.0      1
L-LC       XY25        39198.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 36.0 seconds
